In [1]:
!wget -c https://github.com/heukirne/brazilian-blog-dataset/raw/master/corpus.csv.gz -O corpus.csv.gz

--2019-01-30 23:13:06--  https://github.com/heukirne/brazilian-blog-dataset/raw/master/corpus.csv.gz
Resolving github.com (github.com)... 192.30.255.113, 192.30.255.112
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/heukirne/brazilian-blog-dataset/master/corpus.csv.gz [following]
--2019-01-30 23:13:06--  https://raw.githubusercontent.com/heukirne/brazilian-blog-dataset/master/corpus.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [2]:
import pandas as pd

corpus = pd.read_csv('corpus.csv.gz')
corpus.shape, corpus.columns

((1000, 22),
 Index(['Unnamed: 0', 'id', 'qual_a_melhor_classificao_para_esse_texto',
        'qual_a_melhor_classificao_para_esse_texto:confidence', 'authorid',
        'blogid', 'content', 'contentcount', 'date', 'firstpart', 'labels',
        'me', 'polarity', 'postid', 'published', 'readmore', 'replies',
        'rownum', 'title', 'titlecount', 'wpscount', 'year'],
       dtype='object'))

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk.stem
import nltk

nltk.download('stopwords')
nltk.download('rslp')
stopwords_list = stopwords.words("portuguese")
portuguese_stemmer = nltk.stem.RSLPStemmer()
stopwords_stem = [portuguese_stemmer.stem(w) for w in stopwords_list]

class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer,self).build_analyzer()
        return lambda doc: (portuguese_stemmer.stem(w) for w in analyzer(doc))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [4]:
data = TfidfVectorizer(max_features=1500, strip_accents='unicode', stop_words=stopwords_stem).fit_transform(corpus.content)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ate', 'estiver', 'estivess', 'ha', 'hao', 'houver', 'houvess', 'ja', 'nao', 'sao', 'so', 'tambem', 'tiver', 'tivess'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [0]:
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix = cosine_similarity(data.todense(), data.todense())

In [0]:
import networkx as nx

text_graph = nx.from_numpy_matrix(sim_matrix)
pr = nx.pagerank(text_graph, alpha=0.9, max_iter=1000)